In [1]:
#### IMPORTS

import re
import abc
import operator
import numpy as np
from collections import Counter, defaultdict, namedtuple, deque, abc
from itertools   import (permutations, combinations, chain, cycle, product, islice, 
                         takewhile, zip_longest, starmap, count as count_from)
from functools   import lru_cache, reduce
from heapq import (heappush, heappop, nlargest, nsmallest)

from pprint import pprint as p, pformat as pf
import toolz.curried as t
from tqdm import tqdm_notebook as tq
from dataclasses import dataclass

#### CONSTANTS

alphabet = 'abcdefghijklmnopqrstuvwxyz'
ALPHABET = alphabet.upper()
infinity = float('inf')

#### SIMPLE UTILITY FUNCTIONS

cat = ''.join

def ints(start, end, step=1):
    "The integers from start to end, inclusive: range(start, end+1)"
    return range(start, end + 1, step)

def first(iterable, default=None): 
    "The first item in an iterable, or default if it is empty."
    return next(iter(iterable), default)

def head(iterable, n=5):
    "The first n items in an iterable"
    return tuple(islice(iterable, n))

def tail(iterable, n=1):
    "Skip n items in an iterable"
    return islice(iterable, n, None)

def first_true(iterable, pred=None, default=None):
    """Returns the first true value in the iterable.
    If no true value is found, returns *default*
    If *pred* is not None, returns the first item
    for which pred(item) is true."""
    # first_true([a,b,c], default=x) --> a or b or c or x
    # first_true([a,b], fn, x) --> a if fn(a) else b if fn(b) else x
    return next(filter(pred, iterable), default)

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

def upto(iterable, maxval):
    "From a monotonically increasing iterable, generate all the values <= maxval."
    # Why <= maxval rather than < maxval? In part because that's how Ruby's upto does it.
    return takewhile(lambda x: x <= maxval, iterable)

identity = lambda x: x

def quantify(iterable, pred=bool):
    "Count how many times the predicate is true of an item in iterable."
    return sum(map(pred, iterable))

def multimap(items):
    "Given (key, val) pairs, return {key: [val, ....], ...}."
    result = defaultdict(list)
    for (key, val) in items:
        result[key].append(val)
    return result

def overlapping(iterable, n):
    """Generate all (overlapping) n-element subsequences of iterable.
    overlapping('ABCDEFG', 3) --> ABC BCD CDE DEF EFG"""
    if isinstance(iterable, abc.Sequence):
        yield from (iterable[i:i+n] for i in range(len(iterable) + 1 - n))
    else:
        result = deque(maxlen=n)
        for x in iterable:
            result.append(x)
            if len(result) == n:
                yield tuple(result)
                
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    return overlapping(iterable, 2)

def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

def map2d(fn, grid):
    "Apply fn to every element in a 2-dimensional grid."
    return tuple(mapt(fn, row) for row in grid)

def flatmap(fn, *args):
    "Do a map and a one-level flatten"
    return tuple(chain.from_iterable(map(fn, *args)))

def repeat(n, fn, arg, *args, **kwds):
    "Repeat arg = fn(arg) n times, return arg."
    return nth(repeatedly(fn, arg, *args, **kwds), n)

def repeatedly(fn, arg, *args, **kwds):
    "Yield arg, fn(arg), fn(fn(arg)), ..."
    yield arg
    while True:
        arg = fn(arg, *args, **kwds)
        yield arg
        
def repeatedly1(fn, arg, *args, **kwds):
    "Yield fn(arg), fn(fn(arg)), ..."
    return tail(repeatedly(fn, arg, *args, **kwds))

def compose(f, g): 
    "The function that computes f(g(x))."
    return lambda x: f(g(x))

#### FILE INPUT AND PARSING

def Input(day, line_parser=str.strip, test=False, file_template='data/2020/{}.txt'):
    "For this day's input file, return a tuple of each line parsed by `line_parser`."
    return mapt(line_parser, open(file_template.format(
        f'{day}test' if test else day
    )))

@t.curry
def Tokens(line, sep=','):
    "Splits line into delimited tokens"
    return line.strip().split(sep)

def integers(text): 
    "A tuple of all integers in a string (ignore other characters)."
    return mapt(int, re.findall(r'-?\d+', text))

def digits(number):
    "Tuple of digits in number"
    return mapt(int, str(number))

################ 2-D points implemented using (x, y) tuples

def X(point): return point[0]
def Y(point): return point[1]
def Z(point): return point[2]

origin = (0, 0)
HEADINGS = UP, LEFT, DOWN, RIGHT = (0, -1), (-1, 0), (0, 1), (1, 0)

def turn_right(heading): return HEADINGS[HEADINGS.index(heading) - 1]
def turn_around(heading):return HEADINGS[HEADINGS.index(heading) - 2]
def turn_left(heading):  return HEADINGS[HEADINGS.index(heading) - 3]

def add(A, B): 
    "Element-wise addition of two n-dimensional vectors."
    return mapt(sum, zip(A, B))

def sub(A, B): 
    "Element-wise subtraction of two n-dimensional vectors."
    return tuple(a - b for a, b in zip(A, B))

def neighbors4(point): 
    "The four neighboring squares."
    x, y = point
    return (          (x, y-1),
            (x-1, y),           (x+1, y), 
                      (x, y+1))

def neighbors8(point): 
    "The eight neighboring squares."
    x, y = point 
    return ((x-1, y-1), (x, y-1), (x+1, y-1),
            (x-1, y),             (x+1, y),
            (x-1, y+1), (x, y+1), (x+1, y+1))

def cityblock_distance(P, Q=origin): 
    "Manhatten distance between two points."
    return sum(abs(p - q) for p, q in zip(P, Q))

def distance(P, Q=origin): 
    "Straight-line (hypotenuse) distance between two points."
    return sum((p - q) ** 2 for p, q in zip(P, Q)) ** 0.5

def king_distance(P, Q=origin):
    "Number of chess King moves between two points."
    return max(abs(p - q) for p, q in zip(P, Q))

################ Debugging 

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def grep(pattern, iterable):
    "Print lines from iterable that match pattern."
    for line in iterable:
        if re.search(pattern, line):
            print(line)
            
class Struct:
    "A structure that can have any fields defined."
    def __init__(self, **entries): self.__dict__.update(entries)
    def __repr__(self): 
        fields = ['{}={}'.format(f, self.__dict__[f]) 
                  for f in sorted(self.__dict__)]
        return 'Struct({})'.format(', '.join(fields))

################ A* and Breadth-First Search (tracking states, not actions)

def always(value): return (lambda *args: value)

def Astar(start, moves_func, h_func, cost_func=always(1)):
    "Find a shortest sequence of states from start to a goal state (where h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    Path      = lambda s: ([] if (s is None) else Path(previous[s]) + [s])
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(s)
        for s2 in moves_func(s):
            g = path_cost[s] + cost_func(s, s2)
            if s2 not in path_cost or g < path_cost[s2]:
                heappush(frontier, (g + h_func(s2), s2))
                path_cost[s2] = g
                previous[s2] = s

def bfs(start, moves_func, goals):
    "Breadth-first search"
    goal_func = (goals if callable(goals) else lambda s: s in goals)
    return Astar(start, moves_func, lambda s: (0 if goal_func(s) else 1))

## Day 1

In [10]:
nums = set(Input(1, line_parser=int))
len(nums)

200

In [13]:
def has_2020(n):
    return (2020 - n) in nums
n = first_true(nums, pred=has_2020)
n * (2020 - n)

969024

In [17]:
def has_2020_triplet(pair):
    return (2020 - sum(pair)) in nums
n1, n2 = first_true(combinations(nums, 2), pred=has_2020_triplet)
n1 * n2 * (2020 - n1 - n2)

230057040

## Day 2

In [11]:
TOKENS = re.compile(r'(\d+)-(\d+) (\w): (\w+)')
def toks(line):
    [(least, most, char, pw)] = TOKENS.findall(line)
    return int(least), int(most), char, pw

In [13]:
specs = Input(2, line_parser=toks)
specs[:2]

((6, 10, 's', 'snkscgszxsssscss'), (6, 7, 'b', 'bbbbbxkb'))

In [15]:
def is_valid_pw(spec):
    least, most, char, pw = spec
    return least <= pw.count(char) <= most

In [17]:
quantify(specs, is_valid_pw)

414

In [23]:
from operator import xor

def is_valid_pw2(spec):
    pos1, pos2, char, pw = spec
    return xor(pw[pos1 - 1] == char, pw[pos2 - 1] == char)

In [24]:
quantify(specs, is_valid_pw2)

413